In [1]:
def make_features_dic(fname):
    with open(fname) as file:
        return {line.rstrip('\n'): i for i, line in enumerate(file)}

In [2]:
def make_queue(fname, features_dic):
    with open(fname) as file:
        lines = file.readlines()
        queue_x = np.zeros([len(lines), len(features_dic)])
        queue_y = np.zeros([len(lines), 4]) # One-hot表現: [b, t, e, m]
        
        for i, line in enumerate(lines):
            import re
            from nltk.corpus import stopwords
            import snowballstemmer
            
            line = line.rstrip('\n')
            category = line.strip('\t')[0]
            title = line.split('\t')[1]
            
            # queue_xの処理
            # 文字種の統一
            title = title.lower()
            
            # 数字の置き換え -> 除去
            title = re.sub(r'[0-9]+', '', title)
            
            # '-'を' 'に変換
            title = title.replace('-', ' ')
            
            words = title.split()
            
            # ストップワードの除去
            stop_words = stopwords.words('english')
            words2 = [word for word in words if word not in stop_words]
            words = words2
            
            # ステミング処理
            stemmer = snowballstemmer.stemmer('english')
            words2 = [stemmer.stemWord(word) for word in words]
            words = words2
            
            # 記号の除去
            words2 = [word for word in words if word.islower()]
            words = words2
            
            for word in words:
                if word in features_dic.keys():
                    queue_x[i, features_dic[word]] = 1
                    
            # queue_yの処理
            if category == 'b':
                queue_y[i, 0] = 1
            elif category == 't':
                queue_y[i, 1] = 1
            elif category == 'e':
                queue_y[i, 2] = 1
            elif category == 'm':
                queue_y[i, 3] = 1
                
    return queue_x, queue_y

In [3]:
def softmax(queue): # ソフトマックス関数
    return np.exp(queue) / np.sum(np.exp(queue), axis = 1)[:, np.newaxis]

In [4]:
def make_confusion_matrix(queue_x, queue_y, queue_w, vector_b):
    pred_queue_y = np.zeros(queue_y.shape)
    phi = softmax(np.dot(queue_x, queue_w) + vector_b)
    pred_class_labels = np.argmax(phi, axis = 1)
    class_labels = np.argmax(queue_y, axis = 1)
    confusion_matrix = np.zeros([4, 4])
    
    for i in range(len(queue_y)):
        confusion_matrix[class_labels[i]][pred_class_labels[i]] += 1
            
    return confusion_matrix

In [5]:
def get_confusion_matrix(fname):
    features_dic = make_features_dic('train.feature.txt')
    queue_x, queue_y = make_queue(fname, features_dic)
    w_and_b = np.load('results/52_result.npy')
    queue_w = w_and_b[:-1]
    vector_b = w_and_b[-1]
    confusion_matrix = make_confusion_matrix(queue_x, queue_y, queue_w, vector_b)
    
    return confusion_matrix

In [6]:
import numpy as np

train_confusion_matrix = get_confusion_matrix('train.txt')
test_confusion_matrix = get_confusion_matrix('test.txt')

np.save('results/55_result_train', train_confusion_matrix)
np.save('results/55_result_test', test_confusion_matrix)